In [1]:
## This Cell is related to date preprocessing and is completely copied from original kipf's github code!

import numpy as np
import pandas as pd
import scipy.sparse as sp
import torch,torch.nn,torch.sparse,torch.nn.functional,torch.distributions
from sklearn.metrics import roc_auc_score,average_precision_score
from input_data import load_data
from preprocessing import preprocess_graph, sparse_to_tuple, mask_test_edges
np.set_printoptions(threshold=np.inf)





W_gene      =  pd.read_table( "GBM_Adjacency_matrices/gene.txt", delim_whitespace=True).to_numpy()
W_methy     =  pd.read_table( "GBM_Adjacency_matrices/methy.txt", delim_whitespace=True).to_numpy()
W_mirna     =  pd.read_table( "GBM_Adjacency_matrices/mirna.txt", delim_whitespace=True).to_numpy()

gene      =  pd.read_table( "GBM/GLIO_Gene_Expression.txt", delim_whitespace=True).to_numpy().transpose()
methy     =  pd.read_table( "GBM/GLIO_Methy_Expression.txt", delim_whitespace=True).to_numpy().transpose()
mirna     =  pd.read_table( "GBM/GLIO_Mirna_Expression.txt", delim_whitespace=True).to_numpy().transpose()
truelabel =  pd.read_table( "GBM/GLIO_Survival.txt", delim_whitespace=True).to_numpy()




order = W_argsort.reshape(-1)-1

gene = gene[order]
methy = methy[order]
mirna = mirna[order]
truelabel = truelabel[order]

print(gene.shape)
print(methy.shape)
print(mirna.shape)

def get_nomralized_laplacian(a):
    b = np.diag(np.power(1.0 / a.sum(axis=1), 0.5))
    return (b @ a @ b)




# Some preprocessing
adj = W_mirna
features = np.c_[gene,methy]
adj_norm = get_nomralized_laplacian(adj)


num_features = features.shape[1]

#pos_weight = float(adj.shape[0] * adj.shape[0] - adj.sum()) / adj.sum()
#norm = adj.shape[0] * adj.shape[0] / float((adj.shape[0] * adj.shape[0] - adj.sum()) * 2)

pos_weight = 1
norm = 1







(215, 12042)
(215, 1305)
(215, 534)


In [2]:
#Model definition Cell


adj_label_tensor = torch.Tensor(adj)
adj_norm_tensor  = torch.Tensor(adj_norm)
features_tensor  = torch.Tensor(features)

# from the previous cell following arrays are obtained : 
#adj_label
#adj_norm
#features
#val_edges_false
#test_edges
#test_edges_false




class Model(torch.nn.Module):

    def __init__(self,first_layer_dim=1000,embedding_dim=100,A_tilda=None ,**kwargs):
        super(Model, self).__init__(**kwargs)


        self.W0 = torch.nn.Linear(num_features, first_layer_dim)
        self.W1_mean = torch.nn.Linear(first_layer_dim, embedding_dim)
        self.W1_log_std  = torch.nn.Linear(first_layer_dim, embedding_dim)

        self.A_tilda = A_tilda

        self.normal_dist = torch.distributions.MultivariateNormal(loc=torch.zeros(embedding_dim),scale_tril=torch.eye(embedding_dim))

        self.optimizer = torch.optim.Adam(params=list(self.W0.parameters())+list(self.W1_mean.parameters())+list(self.W1_log_std.parameters()),lr=0.01)
        self.recon = None
        

    def train(self,x,A):


        first_layer = torch.nn.functional.relu(torch.matmul(self.A_tilda,self.W0(x)))
        z_mean      = torch.matmul(self.A_tilda,self.W1_mean(first_layer))
        z_log_std   = torch.matmul(self.A_tilda,self.W1_log_std(first_layer))


        z = z_mean +self.normal_dist.sample((x.shape[0],)) * torch.exp(z_log_std)
        
       

        recon = torch.matmul(z,z.transpose(dim0=0,dim1=1))
        self.recon = torch.sigmoid(recon)

        recon_loss = (  A    *    torch.nn.functional.logsigmoid(recon)  ).mean()*pos_weight + ( (1-A) * torch.nn.functional.logsigmoid(-recon)).mean()
        recon_loss = -norm*recon_loss

        kl_loss =   torch.mean(torch.sum(1 + 2 * z_log_std - z_mean**2 -torch.exp(z_log_std)**2, 1))
        kl_loss = -(0.5 / x.shape[0])*kl_loss

        loss = recon_loss+kl_loss


        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        return loss.item()


    




In [3]:
#Training Cell 

my_model = Model(first_layer_dim=30,embedding_dim=15,A_tilda=adj_norm_tensor)


for epoch_counter in range(200):
    
    epoch_loss = my_model.train(features_tensor,adj_label_tensor)
    
    print("Epoch:", '%04d' % (epoch_counter + 1), "train_loss=", "{:.5f}".format(epoch_loss))


print('\n',"#########################################################",'\n')






Epoch: 0001 train_loss= 1.86629
Epoch: 0002 train_loss= 15.71302
Epoch: 0003 train_loss= 5.66508
Epoch: 0004 train_loss= 2.94591
Epoch: 0005 train_loss= 1.88435
Epoch: 0006 train_loss= 1.35395
Epoch: 0007 train_loss= 1.19873
Epoch: 0008 train_loss= 1.11769
Epoch: 0009 train_loss= 1.18130
Epoch: 0010 train_loss= 1.04280
Epoch: 0011 train_loss= 0.96059
Epoch: 0012 train_loss= 0.94309
Epoch: 0013 train_loss= 0.95453
Epoch: 0014 train_loss= 1.03564
Epoch: 0015 train_loss= 1.01453
Epoch: 0016 train_loss= 0.97239
Epoch: 0017 train_loss= 0.95377
Epoch: 0018 train_loss= 0.94351
Epoch: 0019 train_loss= 0.93497
Epoch: 0020 train_loss= 0.93949
Epoch: 0021 train_loss= 0.92608
Epoch: 0022 train_loss= 0.92471
Epoch: 0023 train_loss= 0.89374
Epoch: 0024 train_loss= 0.87462
Epoch: 0025 train_loss= 0.85780
Epoch: 0026 train_loss= 0.86224
Epoch: 0027 train_loss= 0.84894
Epoch: 0028 train_loss= 0.84985
Epoch: 0029 train_loss= 0.83811
Epoch: 0030 train_loss= 0.83253
Epoch: 0031 train_loss= 0.81845
Epoch: 

In [4]:
print(my_model.recon)

tensor([[0.8781, 0.6112, 0.4715,  ..., 0.6196, 0.5075, 0.3729],
        [0.6112, 0.7133, 0.4720,  ..., 0.6187, 0.5530, 0.5472],
        [0.4715, 0.4720, 0.7687,  ..., 0.5776, 0.4099, 0.3930],
        ...,
        [0.6196, 0.6187, 0.5776,  ..., 0.7642, 0.5344, 0.5138],
        [0.5075, 0.5530, 0.4099,  ..., 0.5344, 0.9167, 0.4891],
        [0.3729, 0.5472, 0.3930,  ..., 0.5138, 0.4891, 0.7880]],
       grad_fn=<SigmoidBackward>)


In [5]:
np.savetxt("gvae_fusion_results/res.txt",my_model.recon.detach().numpy(),delimiter='\t')